# ScavToMarkdown: Conversor de PDF para Markdown

O `ScavToMarkdown` é uma solução robusta para transcrever arquivos PDF em texto Markdown, utilizando modelos OpenAI (apenas `gpt-4.1` e `gpt-4o`) via LangChain.

- **Entrada:** caminho de arquivo, bytes, URL ou string base64 do PDF.
- **Saída:** string em Markdown com a transcrição detalhada do PDF.
- **Destaques:** processamento paralelo, logging dinâmico e configurável, integração com LangChain e OpenAI, e tracking detalhado de tokens.

---

## Parâmetros da Classe

| Parâmetro      | Tipo    | Default    | Descrição                                                                 |
|---------------|---------|------------|--------------------------------------------------------------------------|
| model         | str     | 'gpt-4.1'  | Modelo OpenAI a ser utilizado ('gpt-4.1' ou 'gpt-4o')                    |
| temperature   | float   | 0          | Temperatura para geração de texto                                         |
| max_tokens    | int     | 2048       | Número máximo de tokens por resposta                                     |
| timeout       | int     | 90         | Timeout (segundos) para requisições                                      |
| max_workers   | int     | 10         | Número máximo de threads para processamento paralelo                      |
| verbose       | bool    | True       | Ativa/desativa logs detalhados                                           |
| log_level     | str     | 'INFO'     | Nível de log ('DEBUG', 'INFO', 'WARNING', 'ERROR')                       |
| callbacks     | list    | None       | Callbacks do LangChain para tracing                                      |
| logger        | Logger  | None       | Logger customizado (opcional, para integração dinâmica)                  |

**Exceções:**
- `ModelIncompatibilityError`: se o modelo não for 'gpt-4.1' ou 'gpt-4o'.
- `RuntimeError`: problemas ao ler o PDF.

---

In [1]:
# Instalação das dependências (caso necessário)
# !pip install langchain_openai langchain_core openai python-dotenv requests pymupdf tiktoken


## Instanciando a classe

Abaixo, instanciamos o conversor com o modelo padrão e logging ativado.
Você pode passar um logger customizado para integração dinâmica com outros sistemas de logging.


In [2]:
from scav_switch.converters.pdf import ScavToMarkdown
import logging

scav = ScavToMarkdown(model='gpt-4.1', verbose=True)


2025-06-14 03:23:37,809 - ScavToMarkdown - INFO - ScavToMarkdown initialized - Project: None


## Exemplo 1: Converter PDF a partir de um caminho de arquivo

O método `dig` aceita o caminho para um arquivo PDF local.


In [3]:
import os
base_path = os.getcwd()
pdf_path = os.path.join(base_path, 'sample-tables.pdf')
markdown_text = scav.dig(pdf_path)

with open('markdown_text.md', 'w') as f:
    f.write(markdown_text)

print(markdown_text[:250])


2025-06-14 03:23:37,834 - ScavToMarkdown - INFO - Starting PDF to Markdown conversion process
2025-06-14 03:23:37,836 - ScavToMarkdown - INFO - PDF loaded successfully - 145494 bytes
2025-06-14 03:23:38,302 - ScavToMarkdown - INFO - Extracted 11 pages from PDF
2025-06-14 03:23:38,303 - ScavToMarkdown - INFO - Starting parallel processing with 10 workers
2025-06-14 03:23:57,133 - ScavToMarkdown - INFO - Conversion completed - 17058 characters generated



Design and build  
accessible PDF tables  
Sample tables

Table 1

| Column header (TH) | Column header (TH) | Column header (TH) |
|--------------------|--------------------|--------------------|
| **Row header (TH)** | Data cell (TD)     | Data ce


## Exemplo 2: Converter PDF a partir de uma URL

Também é possível informar uma URL direta para o PDF.


In [4]:
pdf_url = 'https://datascav-public-media-bkt.s3.sa-east-1.amazonaws.com/pdf/sample-tables.pdf'
markdown_text = scav.dig(pdf_url)
print(markdown_text[:250])


2025-06-14 03:23:57,139 - ScavToMarkdown - INFO - Starting PDF to Markdown conversion process
2025-06-14 03:23:57,260 - ScavToMarkdown - INFO - PDF loaded successfully - 145494 bytes
2025-06-14 03:23:57,728 - ScavToMarkdown - INFO - Extracted 11 pages from PDF
2025-06-14 03:23:57,728 - ScavToMarkdown - INFO - Starting parallel processing with 10 workers
2025-06-14 03:24:13,395 - ScavToMarkdown - INFO - Conversion completed - 16784 characters generated



Design and build  
accessible PDF tables  
Sample tables

Table 1

| Column header (TH) | Column header (TH) | Column header (TH) |
|--------------------|--------------------|--------------------|
| **Row header (TH)** | Data cell (TD)      | Data c


## Exemplo 3: Converter PDF a partir de uma string base64

Se você possui o PDF em base64, basta passar a string diretamente.


In [5]:
import base64
pdf_base64 = base64.b64encode(open('sample-tables.pdf', 'rb').read()).decode('utf-8')
markdown_text = scav.dig(pdf_base64)
print(markdown_text[:250])


2025-06-14 03:24:13,402 - ScavToMarkdown - INFO - Starting PDF to Markdown conversion process
2025-06-14 03:24:13,403 - ScavToMarkdown - INFO - PDF loaded successfully - 145494 bytes
2025-06-14 03:24:13,854 - ScavToMarkdown - INFO - Extracted 11 pages from PDF
2025-06-14 03:24:13,855 - ScavToMarkdown - INFO - Starting parallel processing with 10 workers
2025-06-14 03:24:28,520 - ScavToMarkdown - INFO - Conversion completed - 16911 characters generated



Design and build  
accessible PDF tables  
Sample tables

Table 1

| **Column header (TH)** | **Column header (TH)** | **Column header (TH)** |
|------------------------|------------------------|------------------------|
| **Row header (TH)**    | D


---

## Tracking detalhado de tokens

Após o processamento, você pode acessar o atributo `tokens_usage` para obter um relatório detalhado de tokens consumidos:


- `input`: tokens de entrada acumulados
- `output`: tokens de saída acumulados
- `total`: tokens totais
- `details`: lista detalhada por página (com fallback se aplicável)


In [6]:
for key, value in scav.tokens_usage.items():
    if key != 'details':
        print(f"{key.capitalize()} Tokens: {value}")

Input Tokens: 3333
Output Tokens: 14875
Total Tokens: 18208


## Boas práticas e observações

- **Modelos suportados:** apenas 'gpt-4.1' e 'gpt-4o'.
- **Entrada flexível:** caminho, URL, bytes ou base64.
- **Logs:** use `verbose=True` para depuração detalhada ou passe um logger customizado.
- **Erros:** utilize try/except para capturar `ModelIncompatibilityError` e outros problemas de entrada.
- **Performance:** para grandes PDFs, ajuste `max_workers` conforme sua infraestrutura.

---

**Exemplo de tratamento de erro:**


In [7]:
from scav_switch.converters.pdf.exceptions import ModelIncompatibilityError

try:
    scav = ScavToMarkdown(model='gpt-3.5-turbo')  # Não suportado
except ModelIncompatibilityError as e:
    print(f'Erro: {e}')


Erro: Model 'gpt-3.5-turbo' is not compatible. Only 'gpt-4.1' and 'gpt-4o' are supported for PDF to Markdown conversion.
